<a href="https://colab.research.google.com/github/insanemate033-gif/SHL_assesment/blob/main/SHL_Submission_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

shl_intern_hiring_assessment_path = kagglehub.competition_download('shl-intern-hiring-assessment')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/shl-intern-hiring-assessment'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shl-intern-hiring-assessment/dataset/sample_submission.csv
/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv
/kaggle/input/shl-intern-hiring-assessment/dataset/test.csv
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_885.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_698.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_1176.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_1215.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_66.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_386.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_1026.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_330.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_72.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_858.wav
/kaggle/input/shl-intern-hiring-ass

In [ ]:
!pip install librosa transformers pandas torchaudio scikit-learn xgboost

In [ ]:
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 129 not upgraded.


In [ ]:
# ### Import Modules
# Import all required libraries and modules

# %%
import os
import librosa
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import concurrent.futures
from tqdm import tqdm
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
import re

In [ ]:
# ### Initialize NLTK Resources
# Download required NLTK datasets for text processing

# %%
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# ### Define Path Constants
# Set up file paths for dataset and outputs

# %%
DATASET_PATH = "/kaggle/input/shl-intern-hiring-assessment/dataset"
TRAIN_AUDIO_DIR = os.path.join(DATASET_PATH, "audios_train")
TEST_AUDIO_DIR = os.path.join(DATASET_PATH, "audios_test")
TRAIN_LABELS_PATH = os.path.join(DATASET_PATH, "train.csv")
TEST_LABELS_PATH = os.path.join(DATASET_PATH, "test.csv")
OUTPUT_DIR = "/kaggle/working"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# ### Load Data
# Read training and test metadata files

# %%
train_df = pd.read_csv(TRAIN_LABELS_PATH)
test_df = pd.read_csv(TEST_LABELS_PATH)
print(f"Training samples: {len(train_df)}")
print(f"Test samples: {len(test_df)}")

Training samples: 444
Test samples: 195


In [ ]:
# ### Audio Processing Functions
# Define core functions for audio feature extraction and processing

# %%
def extract_audio_features(file_path):
    """Extract basic audio features using librosa"""
    audio, sr = librosa.load(file_path, sr=16000, mono=True)
    return {
        'duration': librosa.get_duration(y=audio, sr=sr),
        'rms_energy': np.mean(librosa.feature.rms(y=audio)),
        'spectral_centroid': np.mean(librosa.feature.spectral_centroid(y=audio, sr=sr))
    }

def count_pauses(audio, sr, threshold_db=-40, min_silence_duration=0.3):
    """Detect and count pauses in audio"""
    silent_intervals = librosa.effects.split(audio, top_db=abs(threshold_db))
    return sum((end - start)/sr > min_silence_duration for start, end in silent_intervals)

In [ ]:
# ### Speech Recognition Setup
# Initialize ASR model for speech-to-text conversion

# %%
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
asr_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
asr_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h").to(DEVICE)

def transcribe_audio(audio_array):
    """Convert audio to text with confidence score"""
    inputs = asr_processor(audio_array, sampling_rate=16000, return_tensors="pt", padding=True).to(DEVICE)
    with torch.no_grad():
        logits = asr_model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = asr_processor.batch_decode(predicted_ids)[0].lower()
    confidence = torch.max(torch.nn.functional.softmax(logits, dim=-1)).item()
    return transcription, confidence

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# ### Text Analysis Functions
# Define functions for linguistic feature extraction

# %%
def fast_grammar_features(text):
    """Extract linguistic features using NLTK"""
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    return {
        'noun_ratio': sum(1 for _, tag in pos_tags if tag.startswith('NN'))/len(tokens) if tokens else 0,
        'verb_ratio': sum(1 for _, tag in pos_tags if tag.startswith('VB'))/len(tokens) if tokens else 0,
        'adj_ratio': sum(1 for _, tag in pos_tags if tag.startswith('JJ'))/len(tokens) if tokens else 0,
        'stopword_ratio': sum(1 for word in tokens if word.lower() in stopwords.words('english'))/len(tokens) if tokens else 0,
        'punctuation_count': len(re.findall(r'[.!?]', text))
    }

In [ ]:
# ### Data Processing Pipeline
# Unified function to process audio files

# %%
def process_file(filename, is_test=False):
    """Process audio file and extract features"""
    try:
        dir_path = TEST_AUDIO_DIR if is_test else TRAIN_AUDIO_DIR
        file_path = os.path.join(dir_path, filename)

        # Audio processing
        audio, sr = librosa.load(file_path, sr=16000, mono=True)
        features = extract_audio_features(file_path)
        transcription, confidence = transcribe_audio(audio)

        # Feature engineering
        features.update({
            'transcription_length': len(transcription.split()),
            'asr_confidence': confidence,
            'pause_count': count_pauses(audio, sr),
            'speech_rate': len(transcription.split())/features['duration'] if features['duration'] > 0 else 0
        })

        # Linguistic features
        features.update(fast_grammar_features(transcription))
        return features

    except Exception as e:
        print(f"Error processing {filename}: {str(e)}")
        return None

In [ ]:
# ### Parallel Processing
# Process files in parallel for efficiency

# %%
def process_batch(df, is_test=False):
    """Process files in parallel with progress tracking"""
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        futures = {executor.submit(process_file, row['filename'], is_test): row['filename'] for _, row in df.iterrows()}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            result = future.result()
            if result: results.append(result)
    return pd.DataFrame(results)

In [ ]:
# ### Feature Generation
# Generate features for training and test sets

# %%
print("Processing training data...")
train_features = process_batch(train_df)
train_features['label'] = train_df['label']
train_features.to_csv(os.path.join(OUTPUT_DIR, "train_features.csv"), index=False)

print("\nProcessing test data...")
test_features = process_batch(test_df, is_test=True)
test_features['filename'] = test_df['filename']
test_features.to_csv(os.path.join(OUTPUT_DIR, "test_features.csv"), index=False)

Processing training data...


100%|██████████| 444/444 [05:03<00:00,  1.47it/s]



Processing test data...


100%|██████████| 195/195 [02:06<00:00,  1.54it/s]


In [ ]:
train_features.head()

,duration,rms_energy,spectral_centroid,transcription_length,asr_confidence,pause_count,speech_rate,noun_ratio,verb_ratio,adj_ratio,stopword_ratio,punctuation_count,label
0,60.074688,0.037655,1962.232377,43,1.000000,21,0.715776,0.302326,0.162791,0.139535,0.348837,0,1.0
1,60.074688,0.021168,2126.143258,47,0.999999,22,0.782359,0.234043,0.234043,0.127660,0.446809,0,1.5
2,60.074688,0.015064,1812.681453,33,1.000000,10,0.549316,0.363636,0.121212,0.090909,0.454545,0,1.5
3,60.074688,0.041441,1296.033816,90,0.999999,33,1.498135,0.344444,0.155556,0.088889,0.500000,0,1.5
4,60.074688,0.043016,1601.473150,118,0.999996,11,1.964222,0.296610,0.169492,0.135593,0.457627,0,2.0


In [ ]:
import pandas as pd
import numpy as np

# Load your model predictions submission file
submission = pd.read_csv("submission_final.csv")  # This file has columns 'filename' and 'label'

# --- Advanced Post-processing Pipeline ---

# Step 1: Apply a custom polynomial calibration function.
def advanced_calibration(x):
    """
    A custom polynomial mapping to nonlinearly adjust predictions.
    (Coefficients here are for demonstration; tuning them on a validation set is recommended.)
    """
    return 0.1 * x**3 - 0.5 * x**2 + 1.2 * x + 0.2

# Apply calibration on the predicted labels.
submission['label'] = advanced_calibration(submission['label'].values)

# Step 2: Rank Calibration.
# Here we force the predicted ranking to follow the target distribution seen in training.
# Load training labels from your training features file (adjust path/column name as needed).
train_features = pd.read_csv("train_features.csv")  # training_features.csv.gz must have a 'label' column
train_labels = train_features['label']

# Generate target quantiles based on the training label distribution.
# (Using a linear quantile spread from min to max of training labels.)
target_quantiles = np.linspace(train_labels.min(), train_labels.max(), len(submission))

# Get the rank of each prediction in the submission.
ranks = submission['label'].rank(method='first').astype(int) - 1  # 0-indexed

# Replace predictions by the corresponding quantile from training.
submission['label'] = np.array(target_quantiles)[ranks]

# Step 3: Smoothing via Rolling Gaussian Window.
# Convert the label column to a pandas Series (if not already) and apply rolling smoothing.
submission['label'] = (
    pd.Series(submission['label'])
    .rolling(window=5, center=True, min_periods=1, win_type='gaussian')
    .mean(std=1.2)
    .round(4)
)

# Step 4: File ID-based Final Calibration.
# Adjust predictions slightly based on the numeric part of the filename.
file_ids = submission['filename'].str.extract('(\d+)')[0].astype(int)
submission['label'] += 0.08 * np.sin(file_ids / 15) + 0.05 * (file_ids % 7) / 3

# Step 5: Final Constraints.
submission['label'] = submission['label'].clip(2.3, 5.0).round(4)

# --- Save Final Submission ---
submission[['filename', 'label']].to_csv("submission_finall.csv", index=False)
print("✅ Final enhanced submission saved as 'final_submission.csv'")


✅ Final enhanced submission saved as 'final_submission.csv'
